In [1]:
import pandas as pd

### TODO:
* sklearn text
* spaCy
* transformers
* Anomaly detection in NLP
### 19.04
* Бейзлайн за 2 часа
* Полный pipeline для обучения за 2 часа
* Ресерч техник, нейронки
* Исследовать метрику (чем плоха f1)
####
Умные мысли
* насколько числа играют важную роль?
* играет ли роль стемминг?
* играет ли роль нормализация?

In [2]:
train_df = pd.read_csv("../dataset/train.tsv", sep="\t")
test_df = pd.read_csv("../dataset/train.tsv", sep="\t")

# Preprcessing
## TODO:
* lower()
* remove punkt
* change digits to words

In [26]:
import string
from nltk.corpus import stopwords
from num2words import num2words

def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    STOPWORDS = stopwords.words('russian')
    ext_punt = string.punctuation + "«" + "»"
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in ext_punt]
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)

    tokens = []
    for token in nopunc.split(" "):
        value = token
        try:
            value = int(token)
            value = num2words(value, lang='ru')
        except ValueError:
            pass
        finally:
            tokens.append(value)

    nopunc = " ".join(tokens)
    # Now just remove any stopwords
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

In [27]:
train_df['clean_title'] = train_df['title'].apply(text_process)
# есть ли смысл убирать римские цифры и тд?

In [28]:
train_df['clean_title']

0       Москвичу Владимиру Клутину пришёл счёт вмешате...
1       Агент Кокорина назвал езду встречке житейской ...
2       Госдума рассмотрит возможность введения секрет...
3       ФАС заблокировала поставку скоростных трамваев...
4       Против Навального завели дело недоносительстве...
                              ...                        
5753     Эдди Чемберс получил сотрясение мозга бою Кличко
5754    Правительство застроит Россию нефтепродуктопро...
5755    стыдно дедом новый канцлер ФРГ обратился перво...
5756         Туркмения декабре начнет поставки газа Китай
5757    Бывший тренер Локомотива возглавил нальчикский...
Name: clean_title, Length: 5758, dtype: object

# Pipeline

In [39]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
pipe1 = Pipeline([('bow', CountVectorizer()),
                 ('tfid', TfidfTransformer()),
                 ('model', MultinomialNB())])
pipe2 = Pipeline([('bow', CountVectorizer()),
                 ('tfid', TfidfTransformer()),
                 ('model', LogisticRegression())])
pipe3 = Pipeline([('tfid', TfidfVectorizer(strip_accents='unicode', ngram_range=(1, 2))),
                 # ('tfid', ),
                 ('model', MultinomialNB())])

# Vectorizer(Count, tfidf, both), model(Bayes, SVD, LogReg, word2vec*, nn*), * different preprocessing

In [30]:
model_selection.cross_val_score(pipe1, train_df['clean_title'], train_df['is_fake'], cv=cv, n_jobs=-1, scoring='f1')

array([0.829     , 0.82965932, 0.82875686])

In [33]:
model_selection.cross_val_score(pipe2, train_df['clean_title'], train_df['is_fake'], cv=cv, n_jobs=-1, scoring='f1')

array([0.81558442, 0.81515617, 0.81295716])

In [40]:
model_selection.cross_val_score(pipe3, train_df['clean_title'], train_df['is_fake'], cv=cv, n_jobs=-1, scoring='f1')

array([0.82934132, 0.83009467, 0.82194514])